In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
house_data=pd.read_csv("train.csv", index_col='Id')
house_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [8]:
house_data.LotFrontage.value_counts()

60.0     143
70.0      70
80.0      69
50.0      57
75.0      53
        ... 
106.0      1
38.0       1
138.0      1
140.0      1
137.0      1
Name: LotFrontage, Length: 110, dtype: int64

In [9]:
house_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-

In [11]:
cat_columns=list(house_data.select_dtypes('object').columns)
if len(cat_columns)>0:
    print("""Below columns have nan values. They will be filled by "no_value". """)
    print(cat_columns)
    for col in cat_columns:
        house_data[col] = house_data[col].fillna('no_value')
    print("Categorical columns are free of nan values")
else:
    print("Categorical columns are free of nan values")

Below columns have nan values. They will be filled by "no_value". 
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
Categorical columns are free of nan values


In [12]:
house_data['LotFrontage'] = house_data['LotFrontage'].fillna(0) #filling nan lot area with 0
house_data['MasVnrArea'] = house_data['MasVnrArea'].fillna(house_data['MasVnrArea'].mean()) #filling nan with mean area
house_data['GarageYrBlt'] = house_data['GarageYrBlt'].fillna(house_data['GarageYrBlt'].mode()[0]) #filling nan year with most occuring area 
house_data.info()
# All the nan values are taken care of

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 no

In [13]:
from sklearn.preprocessing import OrdinalEncoder
oe=OrdinalEncoder()

In [14]:
house_data[cat_columns] = oe.fit_transform(house_data[cat_columns])

In [15]:
feature_data=house_data.drop(columns=['SalePrice'])
target_data=house_data.SalePrice

from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(feature_data, target_data)

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

lr=LinearRegression()
dr=DecisionTreeRegressor()
rf=RandomForestRegressor(n_estimators=100, n_jobs=3)

In [17]:
lr.fit(trainX,trainY)

LinearRegression()

In [18]:
print("Attained an accuracy of {} using Linear regression".format(lr.score(testX,testY)))

Attained an accuracy of 0.6465062736041036 using Linear regression


In [19]:
dr.fit(trainX,trainY)

DecisionTreeRegressor()

In [20]:
print("Attained an accuracy of {} using Decision tree regressor".format(dr.score(testX,testY)))

Attained an accuracy of 0.6306236274392316 using Decision tree regressor


In [21]:
rf.fit(trainX,trainY)

RandomForestRegressor(n_jobs=3)

In [22]:
print("Attained an accuracy of {} using random forest regressor".format(rf.score(testX,testY)))

Attained an accuracy of 0.8517646354735197 using random forest regressor


In [23]:
from sklearn.feature_selection import VarianceThreshold

vr=VarianceThreshold(threshold=0.2)

vr_feature = vr.fit_transform(feature_data)

In [24]:
trainx, testx, trainy, testy = train_test_split(vr_feature, target_data)
lr.fit(trainx,trainy)
dr.fit(trainx,trainy)
rf.fit(trainx,trainy)
print("Attained an accuracy of {} using Linear regression".format(lr.score(testx,testy)))
print("Attained an accuracy of {} using Decision tree regressor".format(dr.score(testx,testy)))
print("Attained an accuracy of {} using random forest regressor".format(rf.score(testx,testy)))

Attained an accuracy of 0.8558570408816855 using Linear regression
Attained an accuracy of 0.7171414017549979 using Decision tree regressor
Attained an accuracy of 0.8892513650942935 using random forest regressor


In [ ]:
# accuracy increased for Linear Regression and Decision tree

In [25]:
#we are getting an accuracy of 88.9 from random forest. We will use the same to predict test data

test_data = pd.read_csv("test.csv", index_col='Id')
test_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [26]:
cat_columns_test=list(test_data.select_dtypes('object').columns)
if len(cat_columns_test)>0:
    print("""Below columns have nan values. They will be filled by "no_value". """)
    print(cat_columns_test)
    for col in cat_columns_test:
        test_data[col] = test_data[col].fillna('no_value')
    print("Categorical columns are free of nan values")
else:
    print("Categorical columns are free of nan values")

Below columns have nan values. They will be filled by "no_value". 
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
Categorical columns are free of nan values


In [27]:
test_data['LotFrontage'] = test_data['LotFrontage'].fillna(0) #filling nan lot area with 0
test_data['MasVnrArea'] = test_data['MasVnrArea'].fillna(test_data['MasVnrArea'].mean()) #filling nan with mean area
test_data['GarageYrBlt'] = test_data['GarageYrBlt'].fillna(test_data['GarageYrBlt'].mode()[0]) #filling nan year with most occuring area 
test_data.info()
# All the nan values are taken care of

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1461 to 2919
Data columns (total 79 columns):
MSSubClass       1459 non-null int64
MSZoning         1459 non-null object
LotFrontage      1459 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null object
Alley            1459 non-null object
LotShape         1459 non-null object
LandContour      1459 non-null object
Utilities        1459 non-null object
LotConfig        1459 non-null object
LandSlope        1459 non-null object
Neighborhood     1459 non-null object
Condition1       1459 non-null object
Condition2       1459 non-null object
BldgType         1459 non-null object
HouseStyle       1459 non-null object
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null object
RoofMatl         1459 non-null object
Exterior1st      1459 non-null object
Exterior2nd      1459

In [29]:
from sklearn.preprocessing import OrdinalEncoder
oe=OrdinalEncoder()

test_data[cat_columns_test] = oe.fit_transform(test_data[cat_columns_test])

test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1461 to 2919
Data columns (total 79 columns):
MSSubClass       1459 non-null int64
MSZoning         1459 non-null float64
LotFrontage      1459 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null float64
Alley            1459 non-null float64
LotShape         1459 non-null float64
LandContour      1459 non-null float64
Utilities        1459 non-null float64
LotConfig        1459 non-null float64
LandSlope        1459 non-null float64
Neighborhood     1459 non-null float64
Condition1       1459 non-null float64
Condition2       1459 non-null float64
BldgType         1459 non-null float64
HouseStyle       1459 non-null float64
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null float64
RoofMatl         1459 non-null float64
Exterior1st      1459 non-null float64
Exter

In [31]:
rf.predict(test_data)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [34]:
na_test = [i for i in test_data.columns if test_data[i].isnull().any()]
for col in na_test:
    test_data[col] = test_data[col].fillna(test_data[col].mode()[0])
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1461 to 2919
Data columns (total 79 columns):
MSSubClass       1459 non-null int64
MSZoning         1459 non-null float64
LotFrontage      1459 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null float64
Alley            1459 non-null float64
LotShape         1459 non-null float64
LandContour      1459 non-null float64
Utilities        1459 non-null float64
LotConfig        1459 non-null float64
LandSlope        1459 non-null float64
Neighborhood     1459 non-null float64
Condition1       1459 non-null float64
Condition2       1459 non-null float64
BldgType         1459 non-null float64
HouseStyle       1459 non-null float64
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null float64
RoofMatl         1459 non-null float64
Exterior1st      1459 non-null float64
Exter

In [36]:
vr_test_data = vr.fit_transform(test_data)

In [45]:
result = pd.Series(rf.predict(vr_test_data))

In [50]:
result.index = np.arange(1461, 1461+len(result))

1461    109968.06
1462    143319.58
1463    173623.00
1464    161637.42
1465    140518.75
          ...    
2915     74905.19
2916     83283.62
2917    134925.47
2918    108907.68
2919    179407.90
Length: 1459, dtype: float64

In [49]:
result.to_csv("submission_result.csv")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
